# Functions

In [1]:
import networkx as nx
import math
import matplotlib.pyplot as plt
from networkx.algorithms import tree
import pandas as pd
import glob
import os

from lineage import similarity


In [2]:
# Load a directory and return filename:df pairs
def build_df_dict_dir(nb_dir):
    dataset = {}
    for file in glob.glob(nb_dir+'*.csv'):
        csvfile = os.path.basename(file)
        try:
            dataset[csvfile] = pd.read_csv(file, index_col=0)
        except (pd.parser.CParserError, UnicodeDecodeError) as e:
            # Star Wars: encoding="ISO-8859-1"
            #df = pd.read_csv(
            #"http://math-info.hse.ru/f/2015-16/all-py/data/tariff2012.csv",
            #sep=';')
            if(csvfile == 'StarWars.csv'):
                dataset[csvfile] = pd.read_csv(file, encoding="ISO-8859-1", index_col=0)
            elif(csvfile == 'tariff2012.csv'):
                dataset[csvfile] = pd.read_csv(file, sep=";", index_col=0)
            else:
                try:
                    dataset[csvfile] = pd.read_csv(file, encoding="ISO-8859-1")
                except:
                    print("Error reading file:", file)
            
    return dataset

In [3]:
# Pre Clustering Functions

from collections import defaultdict

def exact_schema_cluster(df_dict):
    clusters = defaultdict(list)
    for fname, df in df_dict.items():
        clusters[frozenset(df)].append(fname)
    return clusters

def write_clusters_to_file(clusters, cluster_file):
    with open(cluster_file, 'w') as fp:
        for i, cluster in enumerate(clusters.values()):
            fp.write("%d,%d,%s\n" % (i, len(cluster), ",".join(cluster)))
        

In [4]:
def check_csv_graph(artifact_dir, g_truth):
    missing_files = []
    for node in g_truth.nodes():
        if not os.path.exists(artifact_dir+node):
            print("Missing File: "+artifact_dir+node)
            missing_files.append(node)
    return missing_files

In [106]:
import os
from lineage import graphs, similarity
import pandas as pd

def lineage_inference(wf_dir, pre_cluster=False, index=True, threshold=0.0001):
    
    nb_name = os.path.basename(wf_dir)
    if index:
        artifact_dir = wf_dir+'/artifacts/'
    else:
        artifact_dir = wf_dir+'/artifacts_1/'
    
    result_dir = wf_dir+'/inferred/'
    os.makedirs(result_dir, exist_ok=True)
    schema_file = result_dir+'schema_matching.csv'
    row_file = result_dir+'row_matching.csv'
    cluster_file = result_dir+'clusters.csv'
    
    dataset = build_df_dict_dir(artifact_dir)
    
    # Run the inference
    if pre_cluster:
        pairwise_jaccard = []
        clusters = exact_schema_cluster(dataset)
        for cluster in clusters.values():
            batch = {k: dataset[k] for k in cluster}
            pw_batch = similarity.get_pairwise_similarity(batch, similarity.compute_jaccard_DF, threshold=threshold)
            pairwise_jaccard.extend(pw_batch)
    else:      
        pairwise_jaccard = similarity.get_pairwise_similarity(dataset, similarity.compute_jaccard_DF, threshold=threshold)
    
    
    pw_jaccard_graph = graphs.generate_pairwise_graph(pairwise_jaccard)
    nx.to_pandas_adjacency(pw_jaccard_graph,weight='weight').to_csv(
                                                result_dir+'cell_sim.csv')
    
    g_inferred = graphs.generate_spanning_tree(pw_jaccard_graph)
    nx.write_edgelist(g_inferred,result_dir+'infered_mst_cell.csv',data=True)

      
    # Load Ground Truth:
    g_truth = nx.read_gpickle(wf_dir+'/'+nb_name+'_gt_fixed.pkl')

    missing_files = check_csv_graph(artifact_dir, g_truth)
    
    pr_df = pd.DataFrame(columns = ['nb_name', 'index', 'preclustering', 'distance_metric',
                                        'edges_correct', 'edges_missing', 'edges_to_remove', 
                                        'precision', 'recall', 'F1', 'missing_files' ])
    print('Writing Cluster File')
    
    write_clusters_to_file(exact_schema_cluster(dataset), result_dir+'clusters_with_filename.csv')
    
    #print("Adding Join Edges")
    #join_list = find_all_joins_df_dict(dataset)
    #print(len(join_list), "Join Edges Added")
    #g_inferred = add_join_edges(join_list, g_inferred)
    
    
    #Check Join Precision/Recall
    
    #inferred_j_edges = []
    #for join in join_list:
    #    inferred_j_edges.append((join[0], join[2]))
    #    inferred_j_edges.append((join[1], join[2]))
    
    
    #write_join_candidates(join_list, result_dir+'join_candidates.csv')
    
    g_truth_j_edges = [(u,v) for u,v,d in g_truth.edges(data=True) \
                       if g_truth[u][v]['operation'] == 'merge' ]
    
    #print(get_join_precision_recall(g_truth_j_edges, inferred_j_edges))
    
    result = graphs.get_precision_recall(g_truth,g_inferred)

    pr_df = pr_df.append({
            'nb_name': nb_name,
            'index': index,
            'preclustering': pre_cluster,
            'distance_metric': 'pandas_cell',
            'edges_correct': len(result['correct_edges']),
            'edges_missing': len(result['to_add']),
            'edges_to_remove': len(result['to_remove']),
            'precision': result['Precision'],
            'recall': result['Recall'],
            'F1': result['F1'],
            'missing_files': len(missing_files)
        }, ignore_index=True)
    
    return pr_df

In [107]:
base_dir = '/media/suhail/Data/experiments/reexec/res/'
sample_wf = base_dir+'home-depot'
artifact_dir = sample_wf+'/artifacts/'
#dataset = build_df_dict_dir(artifact_dir)

In [108]:
lineage_inference(sample_wf, pre_cluster=False)

Writing Cluster File


,nb_name,index,preclustering,distance_metric,edges_correct,edges_missing,edges_to_remove,precision,recall,F1,missing_files
0,home-depot,True,False,pandas_cell,11,3,1,0.785714,0.916667,0.846154,0


In [ ]:
sample_wf = '/home/suhail/Projects/relic/primitives/python/generator/dataset/20190802-112245'
lineage_inference(sample_wf, pre_cluster=True)

# Single Notebook Test

In [38]:
pre_cluster_types = [False]
index_types = [True]

# Run for Multiple Notebooks

In [374]:
nb_list = [
    'nb_331056',
    'nb_23457',
    # nb_336256', #
    'nb_33614',
    # 'nb_650868', #
    'nb_316514',
    'nb_386796',
    'nb_266913',
    'nb_417011',
    'nb_269991',
    'nb_495072',
    'nb_315236',
    'nb_484354',
    #'nb_772851',
    #'nb_924102',
    #'nb_921915',
    'nb_986282',
    # 'nb_582525', #
    'nb_639263',
]

#fakerdir = '/home/suhail/Projects/relic/primitives/python/generator/dataset/'

#nb_list = [d for d in os.listdir(fakerdir) if os.path.isdir(os.path.join(fakerdir, d))]
nb_list

['nb_331056',
 'nb_23457',
 'nb_33614',
 'nb_316514',
 'nb_386796',
 'nb_266913',
 'nb_417011',
 'nb_269991',
 'nb_495072',
 'nb_315236',
 'nb_484354',
 'nb_986282',
 'nb_639263']

In [375]:
# Copy nbfiles from source to destination
import shutil
import glob

for f in nb_list:
    full_name = f + '.ipynb'
    src = '/media/suhail/Data/experiments/reexec/' + full_name
    dst = base_dir+full_name+'/'+'artifacts/'
    #print(dst)
    print(len([file for file in glob.glob(dst+'*.csv')]))
    #shutil.copy(src, dst, follow_symlinks=True)

11
7
21
7
7
8
8
7
10
9
20
8
8


In [ ]:
# Collect statistics



In [39]:
nb_list = [
    'nb_331056.ipynb',
    'nb_495072.ipynb',
    'nb_315236.ipynb',
    'churn',
    'githubviz',
    'titanic'
]

In [40]:
from tqdm import tqdm_notebook, tqdm
import glob

all_pr_df = pd.DataFrame(columns = ['nb_name', 'index', 'preclustering', 'distance_metric',
                                        'edges_correct', 'edges_missing', 'edges_to_remove', 
                                        'precision', 'recall', 'F1','missing_files' ])

errors = []

for nb in tqdm_notebook(nb_list, desc='notebook', leave=True):
    nb_dir = ('/media/suhail/Data/experiments/reexec/res/'+nb)
    # print('Processing:', nb_dir)
    os.makedirs(nb_dir+'/inferred', exist_ok=True)

    for cluster in tqdm_notebook(pre_cluster_types, desc='cluster', leave=False):
        for index in tqdm_notebook(index_types,  desc='index', leave=False):
            files = glob.glob(nb_dir+'/inferred/*')
            for f in files:
                os.remove(f)
            try:
                result_df = lineage_inference(nb_dir, index=index, pre_cluster=False)
                all_pr_df = pd.concat([all_pr_df, result_df],ignore_index=True)
            except FileNotFoundError as e:
                errors.append((nb_dir, cluster,index))
                pass
                
                

Writing Cluster File
Adding Join Edges
1 Join Edges Added


Writing Cluster File
Adding Join Edges
1 Join Edges Added


Writing Cluster File
Adding Join Edges
0 Join Edges Added


Writing Cluster File
Adding Join Edges
0 Join Edges Added


Writing Cluster File
Adding Join Edges
8 Join Edges Added


Writing Cluster File
Adding Join Edges
7 Join Edges Added



In [37]:
from tqdm import tqdm_notebook, tqdm
import glob

all_pr_df = pd.DataFrame(columns = ['nb_name', 'index', 'preclustering', 'distance_metric',
                                        'edges_correct', 'edges_missing', 'edges_to_remove', 
                                        'precision', 'recall', 'F1','missing_files' ])

errors = []
threshold = 0.001

for nb in tqdm_notebook(nb_list, desc='notebook', leave=True):
    nb_dir = (base_dir+nb)
    print('Processing:', nb_dir)
    os.makedirs(nb_dir+'/inferred', exist_ok=True)

    for cluster in tqdm_notebook(pre_cluster_types, desc='cluster', leave=False):
        for index in tqdm_notebook(index_types,  desc='index', leave=False):
            files = glob.glob(nb_dir+'/inferred/*')
            for f in files:
                os.remove(f)
            try:
                result_df = lineage_inference(nb_dir, index=index, pre_cluster=True, threshold=threshold)
                all_pr_df = pd.concat([all_pr_df, result_df],ignore_index=True)
            except FileNotFoundError as e:
                errors.append((nb_dir, cluster,index))
                pass
                
                

Processing: /media/suhail/Data/experiments/reexec/res/nb_331056.ipynb



NameError: name 'pre_cluster_types' is not defined

In [41]:
all_pr_df

,nb_name,index,preclustering,distance_metric,edges_correct,edges_missing,edges_to_remove,precision,recall,F1,missing_files
0,nb_331056.ipynb,True,False,pandas_cell,3,7,5,0.300000,0.375000,0.333333,0
1,nb_495072.ipynb,True,False,pandas_cell,6,1,1,0.857143,0.857143,0.857143,0
2,nb_315236.ipynb,True,False,pandas_cell,4,2,1,0.666667,0.800000,0.727273,0
3,churn,True,False,pandas_cell,2,3,0,0.400000,1.000000,0.571429,0
4,githubviz,True,False,pandas_cell,6,0,4,1.000000,0.600000,0.750000,0
5,titanic,True,False,pandas_cell,8,4,7,0.666667,0.533333,0.592593,0


In [311]:
new_dict = {'base_table_size': ['6000x10', '100x10', '1000x10', '184x50','1000x20', '100x20', '100x10'], 'versions' : ['60', '55', '10', '55','24', '58', '24']}
all_pr_df['size'] = new_dict['base_table_size']
all_pr_df['versions'] = new_dict['versions']
all_pr_df[['size', 'versions', 'edges_correct', 'edges_missing', 'edges_to_remove', 'precision', 'recall', 'F1']]

,size,versions,edges_correct,edges_missing,edges_to_remove,precision,recall,F1
0,6000x10,60,24,36,5,0.400000,0.827586,0.539326
1,100x10,55,26,28,10,0.481481,0.722222,0.577778
2,1000x10,10,10,10,5,0.500000,0.666667,0.571429
3,184x50,55,23,31,5,0.425926,0.821429,0.560976
4,1000x20,24,10,13,1,0.434783,0.909091,0.588235
5,100x20,58,21,36,10,0.368421,0.677419,0.477273
6,100x10,24,9,14,3,0.391304,0.750000,0.514286


In [13]:
lineage_inference('/home/suhail/Projects/relic/primitives/python/generator/dataset/20190802-112245', index=False, pre_cluster='no_pre_cluster', threshold=threshold)

Error reading file: /home/suhail/Projects/relic/primitives/python/generator/dataset/20190802-112245/artifacts_1/18.csv


/home/suhail/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.parser is deprecated and will be removed in a future version.
You can access CParserError as pandas.errors.ParserError
  


/home/suhail/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3755: RuntimeWarning: '<' not supported between instances of 'str' and 'float', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)
/home/suhail/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3755: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)
/home/suhail/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3755: RuntimeWarning: '<' not supported between instances of 'float' and 'str', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


/home/suhail/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py:3755: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


,nb_name,index,preclustering,distance_metric,edges_correct,edges_missing,edges_to_remove,precision,recall,F1,missing_files
0,20190802-112245,False,no_pre_cluster,pandas_cell,13,10,4,0.565217,0.764706,0.65,0


In [18]:
from nppo import *
import os
import networkx as nx

def compare_edges(g_truth, g_inferred):
    g_edge_set = set([frozenset((v1, v2)) for v1, v2 in g_truth.edges])
    t_edge_set = set([frozenset((v1, v2)) for v1, v2 in g_inferred.edges])

    correct = g_edge_set.intersection(t_edge_set)
    
    to_add  = g_edge_set - t_edge_set
    to_remove = t_edge_set - g_edge_set
    
    return correct, to_add, to_remove


def get_edge_operation(g_truth, edge):
    # Edge is (u,v) pair
    u,v = edge
    if (u,v) in g_truth.edges:
        return g_truth[u][v]['operation']
    elif (v,u) in g_truth.edges:
        return g_truth[v][u]['operation']

def get_edge_weight(g_weights, edge):
    # Edge is (u,v) pair
    u,v = edge
    if (u,v) in g_weights.edges:
        return g_weights[u][v]['weight']
    elif (v,u) in g_weights.edges:
        return g_weights[v][u]['weight']

 
def get_operational_accuracy(nb_dir):
    nb_name = os.path.basename(nb_dir)
    result_dir = nb_dir+'/inferred/'
    
    g_truth = nx.read_gpickle(nb_dir+'/'+nb_name+'_gt.pkl')
    g_inferred = nx.read_edgelist(result_dir+'infered_mst_cell.csv')
    
    g_weights = nx.from_pandas_adjacency(pd.read_csv(result_dir+'cell_sim.csv', index_col=0)) 
    correct, to_add, to_remove = compare_edges(g_truth, g_inferred)
    
    correct_ops = [get_edge_operation(g_truth,(u,v)) for u,v in correct]
    false_negatives = [((u,v), get_edge_operation(g_truth,(u,v)), get_edge_weight(g_weights, (u,v))) for u,v in to_add]
    false_positives = [((u,v), get_edge_weight(g_weights, (u,v))) for u,v in to_remove]
    
    
    missed_merges = [e for e in g_truth.edges(data=True) if e[2]['operation'] == 'merge']
    
    #get_all_joins_wf(nb_name, nb_dir+'/artifacts/')
        
    return missed_merges


In [29]:
result = get_operational_accuracy('/home/suhail/Projects/relic/primitives/python/generator/dataset/20190802-112317')
result

[('2.csv', '4.csv', {'operation': 'merge'}),
 ('3.csv', '4.csv', {'operation': 'merge'}),
 ('4.csv', '31.csv', {'operation': 'merge'}),
 ('14.csv', '17.csv', {'operation': 'merge'}),
 ('14.csv', '27.csv', {'operation': 'merge'}),
 ('16.csv', '17.csv', {'operation': 'merge'}),
 ('17.csv', '20.csv', {'operation': 'merge'}),
 ('19.csv', '20.csv', {'operation': 'merge'}),
 ('22.csv', '29.csv', {'operation': 'merge'}),
 ('26.csv', '27.csv', {'operation': 'merge'}),
 ('28.csv', '29.csv', {'operation': 'merge'}),
 ('30.csv', '31.csv', {'operation': 'merge'}),
 ('30.csv', '42.csv', {'operation': 'merge'}),
 ('31.csv', '48.csv', {'operation': 'merge'}),
 ('32.csv', '36.csv', {'operation': 'merge'}),
 ('32.csv', '39.csv', {'operation': 'merge'}),
 ('35.csv', '36.csv', {'operation': 'merge'}),
 ('37.csv', '60.csv', {'operation': 'merge'}),
 ('38.csv', '39.csv', {'operation': 'merge'}),
 ('41.csv', '42.csv', {'operation': 'merge'}),
 ('47.csv', '48.csv', {'operation': 'merge'}),
 ('59.csv', '60.cs

In [28]:
from collections import defaultdict
merge_dict = defaultdict(set)

for u,v,op in result:
    merge_dict[u].add(u)
    merge_dict[u].add(v)
    merge_dict[v].add(u)
    merge_dict[v].add(v)

merge_dict

NameError: name 'result' is not defined

In [27]:
join_combos = [u for u in merge_dict.values() if len(u) == 3]
join_combos

NameError: name 'merge_dict' is not defined

In [42]:
df_dict = build_df_dict_dir(base_dir+'nb_315236.ipynb')
#[evaluate_join_triple(combo, df_dict) for combo in join_combos]
#df_dict['1.csv'][['ipv4_public', 'mac_processor']].values



In [6]:
evaluate_join_triple(join_combos[6], df_dict) 

{'31.csv', '47.csv', '48.csv'}
Column Union Match: 48.csv ('31.csv', '47.csv')
Checking column coherency of 31.csv 48.csv
Checking column coherency of 47.csv 48.csv
coherent: ('48.csv', ('31.csv', '47.csv'))
intersection:  ('48.csv', ('31.csv', '47.csv'))


('48.csv', ('31.csv', '47.csv'))

In [11]:
for combo in join_combos:
    evaluate_join_triple(combo, df_dict)

{'27.csv', '17.csv', '14.csv'}
{'27.csv', '26.csv', '14.csv'}
{'19.csv', '17.csv', '20.csv'}
{'28.csv', '22.csv', '29.csv'}
Column Union Match: 29.csv ('28.csv', '22.csv')
Checking column coherency of 28.csv 29.csv
Checking column coherency of 22.csv 29.csv
coherent: ('29.csv', ('28.csv', '22.csv'))
intersection:  ('29.csv', ('28.csv', '22.csv'))
{'30.csv', '31.csv', '42.csv'}
{'30.csv', '41.csv', '42.csv'}
{'31.csv', '47.csv', '48.csv'}
Column Union Match: 48.csv ('31.csv', '47.csv')
Checking column coherency of 31.csv 48.csv
Checking column coherency of 47.csv 48.csv
coherent: ('48.csv', ('31.csv', '47.csv'))
intersection:  ('48.csv', ('31.csv', '47.csv'))
{'39.csv', '36.csv', '32.csv'}
{'35.csv', '36.csv', '32.csv'}
{'39.csv', '38.csv', '32.csv'}
{'59.csv', '37.csv', '60.csv'}
Column Union Match: 60.csv ('59.csv', '37.csv')
Checking column coherency of 59.csv 60.csv
Checking column coherency of 37.csv 60.csv
coherent: ('60.csv', ('59.csv', '37.csv'))
intersection:  ('60.csv', ('59.c

In [7]:


%load_ext memory_profiler



In [7]:
df1 = df_dict['47.csv']
dest = df_dict['48.csv']
df2 = df_dict['31.csv']

#coherent_1 = get_max_coherent_columns(df1,dest)
#coherent_2 = get_max_coherent_columns(df2,dest)
#check_col_group_containment(df2,dest,['ipv4', 'ipv4_private', 'url', 'numerify', 'unix_time', 'bothify', 'suffix_female', 'pydecimal', 'bban'])

In [11]:
s = generate_common_lattice(df2,dest)
s

{'address',
 'bban',
 'boolean',
 'city_suffix',
 'country',
 'credit_card_full',
 'currency_code',
 'date',
 'ipv4_public',
 'ipv6',
 'language_code',
 'mac_platform_token',
 'mac_processor',
 'null_boolean',
 'prefix',
 'pyfloat',
 'pyint',
 'random_digit_x',
 'random_digit_y',
 'random_number',
 'rgb_color',
 'sentence',
 'state_abbr',
 'street_address',
 'uri',
 'uri_page'}

In [ ]:
# TODO: Disambiguate _x and _y as left and right side join columns. 

In [10]:
len([s for s in itertools.combinations(s,26)])

1

In [2]:
all_pr_df

NameError: name 'all_pr_df' is not defined

In [149]:
all_pr_df.sort_values('F1', ascending=False)

,nb_name,index,preclustering,distance_metric,edges_correct,edges_missing,edges_to_remove,precision,recall,F1,missing_files
4,nb_266913.ipynb,True,no_pre_cluster,pandas_cell,5,1,1,0.833333,0.833333,0.833333,0
1,nb_23457.ipynb,True,no_pre_cluster,pandas_cell,4,1,1,0.800000,0.800000,0.800000,0
7,nb_495072.ipynb,True,no_pre_cluster,pandas_cell,5,4,1,0.555556,0.833333,0.666667,0
5,nb_417011.ipynb,True,no_pre_cluster,pandas_cell,4,3,1,0.571429,0.800000,0.666667,0
8,nb_315236.ipynb,True,no_pre_cluster,pandas_cell,5,3,2,0.625000,0.714286,0.666667,0
2,nb_33614.ipynb,True,no_pre_cluster,pandas_cell,9,12,3,0.428571,0.750000,0.545455,0
6,nb_269991.ipynb,True,no_pre_cluster,pandas_cell,2,4,2,0.333333,0.500000,0.400000,0
0,nb_331056.ipynb,True,no_pre_cluster,pandas_cell,4,7,6,0.363636,0.400000,0.380952,0
10,nb_986282.ipynb,True,no_pre_cluster,pandas_cell,2,4,4,0.333333,0.333333,0.333333,0
3,nb_386796.ipynb,True,no_pre_cluster,pandas_cell,1,5,2,0.166667,0.333333,0.222222,0


In [78]:
nonindexed_cell = all_pr_df.loc[(all_pr_df.distance_metric == 'cell')
                        & (all_pr_df['index'] == False)]
nonindexed_col = all_pr_df.loc[(all_pr_df.distance_metric == 'col')
                        & (all_pr_df['index'] == False)]
#nonindexed_cell.to_excel('results_noindex_cell.xlsx')
#nonindexed_col.to_excel('results_noindex_col.xlsx')

In [77]:
#all_pr_df.to_excel('results.xlsx')

In [ ]:
# CommandLine Debugging
'''
/home/suhail/Projects/relic/primitives/cpp/src/pre_clustering/pre_clustering -partial_schema -result /media/suhail/Data/experiments/results/ok/nb_639263.ipynb/inferred/ -schema_file /media/suhail/Data/experiments/results/ok/nb_639263.ipynb/inferred/schema_matching.csv
'''

In [39]:
all_pr_df

,nb_name,index,preclustering,distance_metric,edges_correct,edges_missing,edges_to_remove,precision,recall,F1
0,nb_331056.ipynb,True,no_pre_cluster,pandas_cell,5,7,5,0.416667,0.500000,0.454545
1,nb_331056.ipynb,False,no_pre_cluster,pandas_cell,3,9,3,0.250000,0.500000,0.333333
2,nb_23457.ipynb,True,no_pre_cluster,pandas_cell,4,1,1,0.800000,0.800000,0.800000
3,nb_23457.ipynb,False,no_pre_cluster,pandas_cell,4,1,1,0.800000,0.800000,0.800000
4,nb_33614.ipynb,True,no_pre_cluster,pandas_cell,10,11,3,0.476190,0.769231,0.588235
5,nb_33614.ipynb,False,no_pre_cluster,pandas_cell,7,14,1,0.333333,0.875000,0.482759
6,nb_316514.ipynb,True,no_pre_cluster,pandas_cell,3,5,1,0.375000,0.750000,0.500000
7,nb_316514.ipynb,False,no_pre_cluster,pandas_cell,1,7,2,0.125000,0.333333,0.181818
8,nb_386796.ipynb,True,no_pre_cluster,pandas_cell,1,5,1,0.166667,0.500000,0.250000
9,nb_386796.ipynb,False,no_pre_cluster,pandas_cell,1,5,0,0.166667,1.000000,0.285714


In [40]:
groups = all_pr_df.groupby(pd.cut(all_pr_df.index, range(0,len(all_pr_df), 8)))
groups.max()

,nb_name,index,preclustering,distance_metric,edges_correct,edges_missing,edges_to_remove,precision,recall,F1
"(0, 8]",nb_386796.ipynb,True,no_pre_cluster,pandas_cell,10,14,3,0.800000,0.875,0.800000
"(8, 16]",nb_495072.ipynb,True,no_pre_cluster,pandas_cell,5,5,2,0.833333,1.000,0.833333
"(16, 24]",nb_986282.ipynb,True,no_pre_cluster,pandas_cell,4,22,10,0.500000,0.800,0.615385


In [41]:
best_f1 = all_pr_df.groupby(['nb_name'], sort=False)['F1'].max().to_frame()
best_f1

,F1
nb_name,
nb_331056.ipynb,0.454545
nb_23457.ipynb,0.800000
nb_33614.ipynb,0.588235
nb_316514.ipynb,0.500000
nb_386796.ipynb,0.285714
nb_266913.ipynb,0.833333
nb_417011.ipynb,0.666667
nb_269991.ipynb,0.444444
nb_495072.ipynb,0.666667


In [43]:
idx = all_pr_df.groupby(['nb_name'])['F1'].transform(max) == all_pr_df['F1']
max_filter = all_pr_df.loc[idx]


In [44]:
max_filter = max_filter.loc[max_filter['F1'] != 0]
len(max_filter)

18

In [45]:
top_nb_scores = max_filter.drop(['index','preclustering','distance_metric'],axis=1)
top_nb_scores = top_nb_scores.set_index('nb_name')
top_nb_scores = top_nb_scores.drop_duplicates()
top_nb_scores

,edges_correct,edges_missing,edges_to_remove,precision,recall,F1
nb_name,,,,,,
nb_331056.ipynb,5,7,5,0.416667,0.500000,0.454545
nb_23457.ipynb,4,1,1,0.800000,0.800000,0.800000
nb_33614.ipynb,10,11,3,0.476190,0.769231,0.588235
nb_316514.ipynb,3,5,1,0.375000,0.750000,0.500000
nb_386796.ipynb,1,5,0,0.166667,1.000000,0.285714
nb_266913.ipynb,5,1,1,0.833333,0.833333,0.833333
nb_417011.ipynb,4,3,1,0.571429,0.800000,0.666667
nb_269991.ipynb,2,4,1,0.333333,0.666667,0.444444
nb_495072.ipynb,5,4,1,0.555556,0.833333,0.666667


In [50]:
max_filter['index'].value_counts()

True     11
False     7
Name: index, dtype: int64

In [10]:
write_clusters_to_file(exact_schema_cluster(df_dict),'/home/suhail/Projects/relic/primitives/python/generator/dataset/20190802-112245/inferred/clusters_with_filename.csv')

NameError: name 'df_dict' is not defined

In [71]:
df_dict = build_df_dict_dir(artifact_dir)


In [72]:
clusters=exact_schema_cluster(df_dict)
clusters

defaultdict(list,
            {frozenset({'Unnamed: 0.1',
                        'code',
                        'level',
                        'name',
                        'name_en',
                        'name_es',
                        'name_short_en',
                        'name_short_es',
                        'parent_id'}): ['MexHS.csv', 'MexHS__1.csv'],
             frozenset({'Unnamed: 0.1',
                        'code',
                        'level',
                        'name',
                        'parent_id'}): ['HSdf.csv'],
             frozenset({'code', 'level', 'name', 'parent_id'}): ['HSdf__1.csv',
              'AtlasHS.csv'],
             frozenset({'Unnamed: 0.1',
                        'code',
                        'level_colatlas',
                        'level_mex',
                        'name_colatlas',
                        'name_en',
                        'name_es',
                        'name_mex',
                        '

In [73]:
def find_join_order(combo):
    sizes = {x: len(x) for x in combo}
    if max(sizes.values())==min(sizes.values()):
        return None
    join_dest = list(sizes.keys())[list(sizes.values()).index(max(sizes.values()))] 
    join_sources = tuple(x for x in combo if x is not join_dest)
    
    if join_sources[0].union(join_sources[1]) == join_dest:
        return join_sources, join_dest
    
    return None

def find_join_order_general(combo):
    combo_set = set(combo)
    max_combo = None
    max_col_number = 0
    for join_dest in combo_set:
        join_sources = combo_set - set([join_dest])

        common_cols = set()
        for source in join_sources:
            common_cols = common_cols.union(source)
            
        common_cols = common_cols.intersection(join_dest)
        if not common_cols:
            return None

        if len(common_cols) > max_col_number:
            max_col_number = len(common_cols)
            max_combo = (tuple(join_sources), join_dest)
            
    return tuple(max_combo), len(common_cols)

In [74]:
import itertools

def find_join_schemas(clusters):
    schema_combos = [combo for combo in itertools.combinations(clusters.keys(),3)]
    join_schemas = []
    for x in schema_combos:
        #result = find_join_order(x)
        result = find_join_order_general(x)
        if result:
            join_schemas.append(result)
    return join_schemas

In [75]:
#TODO: Fix this
def find_join_schemas_maximal(clusters):
    schema_combos = [combo for combo in itertools.combinations(clusters.keys(),3)]
    join_schemas = defaultdict(lambda: defaultdict(list))
    for x in schema_combos:
        #result = find_join_order(x)
        #print(type(x), x)
        result = find_join_order_general(x)
        if result:
            combo, val = result
            join_result = combo[1]
            join_schemas[join_result][val].append(combo)
    return join_schemas

def prune_join_schemas(join_schemas):
    pruned_candidates = []
    for join_result in join_schemas.keys():
        max_common_col = max(join_schemas[join_result])
        print(max_common_col)
        pruned_candidates.append(join_schemas[join_result][max_common_col])
    
    return pruned_candidates
        

In [76]:
find_join_schemas_maximal(clusters)

defaultdict(<function __main__.find_join_schemas_maximal.<locals>.<lambda>()>,
            {frozenset({'Unnamed: 0.1',
                        'code',
                        'level',
                        'name',
                        'parent_id'}): defaultdict(list,
                         {5: [((frozenset({'code',
                                        'level',
                                        'name',
                                        'parent_id'}),
                             frozenset({'Unnamed: 0.1',
                                        'code',
                                        'level',
                                        'name',
                                        'name_en',
                                        'name_es',
                                        'name_short_en',
                                        'name_short_es',
                                        'parent_id'})),
                            frozenset({'Unnamed: 0.

In [77]:
join_schemas = find_join_schemas(clusters)

In [78]:
join_schemas = find_join_schemas_maximal(clusters)

In [79]:
join_schemas

defaultdict(<function __main__.find_join_schemas_maximal.<locals>.<lambda>()>,
            {frozenset({'Unnamed: 0.1',
                        'code',
                        'level',
                        'name',
                        'parent_id'}): defaultdict(list,
                         {5: [((frozenset({'code',
                                        'level',
                                        'name',
                                        'parent_id'}),
                             frozenset({'Unnamed: 0.1',
                                        'code',
                                        'level',
                                        'name',
                                        'name_en',
                                        'name_es',
                                        'name_short_en',
                                        'name_short_es',
                                        'parent_id'})),
                            frozenset({'Unnamed: 0.

In [80]:
pruned = prune_join_schemas(join_schemas)

5
6


In [81]:
pruned

[[((frozenset({'code', 'level', 'name', 'parent_id'}),
    frozenset({'Unnamed: 0.1',
               'code',
               'level',
               'name',
               'name_en',
               'name_es',
               'name_short_en',
               'name_short_es',
               'parent_id'})),
   frozenset({'Unnamed: 0.1', 'code', 'level', 'name', 'parent_id'}))],
 [((frozenset({'Unnamed: 0.1', 'code', 'level', 'name', 'parent_id'}),
    frozenset({'Unnamed: 0.1',
               'code',
               'level_colatlas',
               'level_mex',
               'name_colatlas',
               'name_en',
               'name_es',
               'name_mex',
               'name_short_en',
               'name_short_es',
               'parent_id_colatlas',
               'parent_id_mex'})),
   frozenset({'Unnamed: 0.1',
              'code',
              'level',
              'name',
              'name_en',
              'name_es',
              'name_short_en',
              

In [90]:
from collections import defaultdict

def enumerate_join_candidates(join_schemas, clusters, df_dict):
    join_candidates = []
    for schema in join_schemas:
        join_l, join_r = clusters[schema[0][0]], clusters[schema[0][1]]
        join_dest = clusters[schema[1]]
        for jl in join_l:
            for jr in join_r:
                for jd in join_dest:
                    coherent_1 = simple_coherency_check(df_dict[jd],df_dict[jl])
                    coherent_2 = simple_coherency_check(df_dict[jd],df_dict[jr])
                    # Check if the coherent columns generate the output set
                    if coherent_1 and coherent_2:
                        if set(coherent_1).union(set(coherent_2)) == set(df_dict[jd]):
                            if set(coherent_1).intersection(set(coherent_2)): # Check if the intersection is not null
                                join_candidates.append((jl,jr,jd))
    return join_candidates


def enumerate_join_candidates_new(join_schemas, clusters, df_dict):
    candidates = {}
    
    for schema in join_schemas:
        max_join_union_size = 0
        join_candidates = defaultdict(list)
        #print(schema)
        join_l, join_r = clusters[schema[0][0][0]], clusters[schema[0][0][1]]
        join_dest = clusters[schema[0][1]]
        for jl in join_l:
            for jr in join_r:
                for jd in join_dest:
                    coherent_1 = simple_coherency_check(df_dict[jd],df_dict[jl])
                    coherent_2 = simple_coherency_check(df_dict[jd],df_dict[jr])
                    # Check if the coherent columns generate the output set
                    # TODO: Check Maximal column containment and atleast one intersection
                    print(coherent_1, coherent_2)
                    if coherent_1 and coherent_2:
                        #if set(coherent_1).union(set(coherent_2)) == set(df_dict[jd]):
                        #    if set(coherent_1).intersection(set(coherent_2)): # Check if the intersection is not null
                        #        join_candidates.append((jl,jr,jd))
                        
                        union = set(coherent_1).union(set(coherent_2))
                        size = len(union.intersection(set(df_dict[jd])))
                        if size > 0 and size >= max_join_union_size:
                            if set(coherent_1).intersection(set(coherent_2)): # Check if the intersection is not null
                                join_candidates[size].append((jl,jr,jd))
                                max_join_union_size = size
                    
        candidates[schema[0]] = join_candidates[max_join_union_size]
    return candidates

In [91]:
import numpy as np

def check_minimal_extra_values(candidates, df_dict):
    # For each schema pair, check for values that should have been joined but are not
    # present
    best_join_candidates = {}
    for schema, candidate_list in candidates.items():
        #Set Difference
        best_matches = defaultdict(list)
        least_surplus = np.inf
        for combo in candidate_list:
            jl,jr,jd = combo
            coherent_l = simple_coherency_check(df_dict[jd],df_dict[jl])
            coherent_r = simple_coherency_check(df_dict[jd],df_dict[jr])
            
            jlvalset = set(frozenset(u) for u in df_dict[jl][list(coherent_l)].values.tolist())
            jrvalset = set(frozenset(u) for u in df_dict[jr][list(coherent_r)].values.tolist())
            jdlvalset = set(frozenset(u) for u in df_dict[jd][list(coherent_l)].values.tolist())
            jdrvalset = set(frozenset(u) for u in df_dict[jd][list(coherent_r)].values.tolist())

            
            left_size = len(jlvalset - jdlvalset)
            right_size = len(jrvalset - jdrvalset)
            
            total_excess = left_size + right_size
            
            if total_excess <= least_surplus:
                best_matches[total_excess].append((jl,jr,jd))
                least_surplus = total_excess
            
        best_join_candidates[schema] = best_matches[least_surplus]

    return best_join_candidates

In [92]:
import nppo
import csv

def simple_coherency_check(base_df, join_dest_df):
    return nppo.get_max_coherent_columns_1(base_df, join_dest_df)
    
def write_join_candidates(join_candidate_list, filename):
    with open(filename,'w') as fp:
        csv_out = csv.writer(fp)
        for row in join_candidate_list:
            csv_out.writerow(row)
        

In [93]:
#DEBUG

schema = [((frozenset({'random_digit', 'ean8', 'numerify', 'prefix_male', 'pybool', 'url', 'randomize_nb_elements', 'random_number', 'ipv4_public'}), frozenset({'msisdn', 'state', 'invalid_ssn', 'domain_name', 'random_int', 'firefox', 'ssn', 'military_apo', 'military_dpo', 'prefix', 'ascii_free_email', 'password', 'zipcode_in_state', 'catch_phrase', 'numerify', 'unix_partition', 'cryptocurrency_code', 'day_of_week', 'pydecimal', 'ipv4_public', 'pyint', 'credit_card_full', 'ascii_safe_email', 'pyfloat', 'ipv4_private', 'last_name'})), frozenset({'ssn', 'military_apo', 'pyint', 'military_dpo', 'numerify', 'unix_partition', 'domain_name', 'ipv4_private', 'ipv4_public'}))]

schema[0][1]

frozenset({'domain_name',
           'ipv4_private',
           'ipv4_public',
           'military_apo',
           'military_dpo',
           'numerify',
           'pyint',
           'ssn',
           'unix_partition'})

In [94]:
jc = enumerate_join_candidates_new(pruned, clusters, df_dict)
jc.values()

{'code', 'name', 'level'} None
{'code', 'name', 'level'} None
None None
None None
{'Unnamed: 0.1'} None
None None
{'Unnamed: 0.1'} None
None None


dict_values([[], []])

In [257]:
jc_pruned = {k:v for k,v in jc.items() if len(v) > 0}



In [278]:
best_result = check_minimal_extra_values(jc_pruned, df_dict)
[val for val in best_result.values()]

[[('12.csv', '18.csv', '19.csv')],
 [('8.csv', '19.csv', '9.csv')],
 [('2.csv', '19.csv', '12.csv'), ('0.csv', '19.csv', '12.csv')],
 [('6.csv', '22.csv', '23.csv')],
 [('1.csv', '13.csv', '14.csv'), ('16.csv', '13.csv', '14.csv')],
 [('10.csv', '23.csv', '22.csv')]]

In [273]:
final_list = []
for val in best_result.values():
    final_list.extend(val)
    
final_list

[('12.csv', '18.csv', '19.csv'),
 ('8.csv', '19.csv', '9.csv'),
 ('2.csv', '19.csv', '12.csv'),
 ('0.csv', '19.csv', '12.csv'),
 ('6.csv', '22.csv', '23.csv'),
 ('1.csv', '13.csv', '14.csv'),
 ('16.csv', '13.csv', '14.csv'),
 ('10.csv', '23.csv', '22.csv')]

In [260]:
for schema, candidates in jc_pruned.items():
    print(len(candidates), len(best_result[schema]))

2 1
3 1
2 2
9 1
5 2
1 1


In [17]:
def find_all_joins_df_dict(df_dict):
    clusters=exact_schema_cluster(df_dict)
    join_schemas = find_join_schemas_maximal(clusters)
    pruned = prune_join_schemas(join_schemas)
    jc = enumerate_join_candidates_new(pruned, clusters, df_dict)
    jc_pruned = {k:v for k,v in jc.items() if len(v) > 0}
    best_result = check_minimal_extra_values(jc_pruned, df_dict)
    final_list = []
    for val in best_result.values():
        final_list.extend(val)

    return final_list


def add_join_edges(join_list, G):
    for join in join_list:
        G.add_edge(join[0], join[2], weight=0)
        G.add_edge(join[1], join[2], weight=0)
    return G

In [11]:

def get_join_precision_recall(G_truth_edges, T_inferred_edges):
    g_edge_set = set([frozenset((v1, v2)) for v1, v2 in G_truth_edges])
    t_edge_set = set([frozenset((v1, v2)) for v1, v2 in T_inferred_edges])

    correct = g_edge_set.intersection(t_edge_set)
    
    to_add  = g_edge_set - t_edge_set
    to_remove = t_edge_set - g_edge_set
    
    try:
        precision = float(len(correct))/len(g_edge_set)
        recall = float(len(correct))/len(t_edge_set)
        f1 = 2 * ((precision * recall) / (precision + recall))
    except ZeroDivisionError as e:
        precision = 0.0
        recall = 0.0
        f1 = 0.0
    
    return {'Precision': precision,
            'Recall': recall,
            'F1': f1,
            'correct_edges': correct,
            'to_add': to_add,
            'to_remove': to_remove}



In [275]:
write_join_candidates(final_list,'/home/suhail/Projects/relic/primitives/python/generator/dataset/20190802-112245/inferred/join_candidates.csv')